# mtbatchgen

Using the mtbatchgen env (Python 3.7, tf 1.14)

- Will save the models 
- will export model weights and re-save them using tf2.

In [8]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [9]:
import tensorflow as tf
import os

In [10]:
tf.__version__

'1.14.0'

In [12]:
!python --version

Python 3.7.6


In [3]:
import pandas as pd

In [4]:
cluster_config = pd.read_csv("../../../figures_factory/configs/cluster.tsv", sep='\t')
cluster_config

,cluster,colour,description,new_cluster
0,1,#B03743,Fibroblast,1
1,2,#E85F6D,Fibroblast-like,3
2,3,#7C2730,Fibroblast-like,2
3,4,#F0A118,Intermediate,12
4,5,#78A824,Pre-iPSC,13
5,6,#A6D854,Pre-iPSC,14
6,7,#F01D35,Fibroblast-like,5
7,8,#13C4A3,iPSC,15
8,9,#406614,Partially-reprogrammed,10
9,10,#D4B81C,Intermediate,9


## Fold 0

In [5]:
import glob

In [6]:
for i in range(1,16):
    paths = glob.glob("/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/bpnet/models/20210820_chrombpnet_lite/chrombpnet/cluster_idx{}/*.h5".format(i))
    assert(len(paths)==2)
    bias_path = [x for x in paths if "adjusted" in x][0]
    cbp_path = [x for x in paths if "adjusted" not in x][0]
    bias_model = tf.keras.models.load_model(bias_path, compile=False)
    cbp_model = tf.keras.models.load_model(cbp_path, compile=False)
    
    new_name = "c" + str(cluster_config.iloc[i-1]['new_cluster']).lower() + "_" + \
            str(cluster_config.iloc[i-1]['description']).replace('-', '_').lower()
    
    print(i, new_name)
    # write weights to intermediate dir
    if not os.path.exists("./intermediate/fold0/"):
        os.makedirs("./intermediate/fold0/")
    
    bias_model.save_weights("./intermediate/fold0/{}.bias.h5".format(new_name))
    cbp_model.save_weights("./intermediate/fold0/{}.chrombpnet.h5".format(new_name))
    
    if not os.path.exists("./out/tf1.14/fold0"):
        os.makedirs("./out/tf1.14/fold0")
    
    bias_model.save("./out/tf1.14/fold0/{}.bias.h5".format(new_name))
    cbp_model.save("./out/tf1.14/fold0/{}.chrombpnet.h5".format(new_name))
    
    tf.keras.backend.clear_session()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1 c1_fibroblast
2 c3_fibroblast_like
3 c2_fibroblast_like
4 c12_intermediate
5 c13_pre_ipsc
6 c14_pre_ipsc
7 c5_fibroblast_like
8 c15_ipsc
9 c10_partially_reprogrammed
10 c9_intermediate
11 c8_xosk
12 c4_fibroblast_like
13 c6_keratinocyte_like
14 c11_intermediate
15 c7_hosk


## Folds 1-9

In [7]:
for fold in range(1,10):
    for i in range(1,16):
        paths = glob.glob("/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/bpnet/models/20230119_chrombpnet_lite_folds1_9/fold{}/chrombpnet/cluster_idx{}/*.h5".format(fold, i))
        assert(len(paths)==2)
        bias_path = [x for x in paths if "adjusted" in x][0]
        cbp_path = [x for x in paths if "adjusted" not in x][0]
        bias_model = tf.keras.models.load_model(bias_path, compile=False)
        cbp_model = tf.keras.models.load_model(cbp_path, compile=False)

        new_name = "c" + str(cluster_config.iloc[i-1]['new_cluster']).lower() + "_" + \
                str(cluster_config.iloc[i-1]['description']).replace('-', '_').lower()

        print(fold, i, new_name)
        # write weights to intermediate dir
        int_dir = "./intermediate/fold{}/".format(fold)
        if not os.path.exists(int_dir):
            os.makedirs(int_dir)

        bias_model.save_weights(int_dir + "{}.bias.h5".format(new_name))
        cbp_model.save_weights(int_dir + "{}.chrombpnet.h5".format(new_name))

        out_dir = "./out/tf1.14/fold{}/".format(fold)
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        bias_model.save(out_dir + "{}.bias.h5".format(new_name))
        cbp_model.save(out_dir + "{}.chrombpnet.h5".format(new_name))
        
        tf.keras.backend.clear_session()

1 1 c1_fibroblast
1 2 c3_fibroblast_like
1 3 c2_fibroblast_like
1 4 c12_intermediate
1 5 c13_pre_ipsc
1 6 c14_pre_ipsc
1 7 c5_fibroblast_like
1 8 c15_ipsc
1 9 c10_partially_reprogrammed
1 10 c9_intermediate
1 11 c8_xosk
1 12 c4_fibroblast_like
1 13 c6_keratinocyte_like
1 14 c11_intermediate
1 15 c7_hosk
2 1 c1_fibroblast
2 2 c3_fibroblast_like
2 3 c2_fibroblast_like
2 4 c12_intermediate
2 5 c13_pre_ipsc
2 6 c14_pre_ipsc
2 7 c5_fibroblast_like
2 8 c15_ipsc
2 9 c10_partially_reprogrammed
2 10 c9_intermediate
2 11 c8_xosk
2 12 c4_fibroblast_like
2 13 c6_keratinocyte_like
2 14 c11_intermediate
2 15 c7_hosk
3 1 c1_fibroblast
3 2 c3_fibroblast_like
3 3 c2_fibroblast_like
3 4 c12_intermediate
3 5 c13_pre_ipsc
3 6 c14_pre_ipsc
3 7 c5_fibroblast_like
3 8 c15_ipsc
3 9 c10_partially_reprogrammed
3 10 c9_intermediate
3 11 c8_xosk
3 12 c4_fibroblast_like
3 13 c6_keratinocyte_like
3 14 c11_intermediate
3 15 c7_hosk
4 1 c1_fibroblast
4 2 c3_fibroblast_like
4 3 c2_fibroblast_like
4 4 c12_intermediate
